# Demos: Lecture 8

In [1]:
import pennylane as qml
from pennylane import numpy as np
from lecture08_helpers import compute_accuracy
from lecture08_helpers import *
import matplotlib.pyplot as plt

# Loads the training and testing data for you
train_X = np.load("data/l8_train_X.npy")
train_y = np.load("data/l8_train_y.npy")
test_X = np.load("data/l8_test_X.npy")
test_y = np.load("data/l8_test_y.npy")

train_X = np.array(train_X, requires_grad=False)
train_y = np.array(train_y, requires_grad=False)
test_X = np.array(test_X, requires_grad=False)
test_y = np.array(test_y, requires_grad=False)

In [2]:
np.set_printoptions(suppress=True)

In [3]:
test_X

tensor([[-0.07161386, -0.22872007,  0.88470143, -0.10581928],
        [ 1.64567103, -0.66481804, -0.57328593,  0.07178712],
        [ 0.26837931,  0.42790794,  0.15068389, -0.45816579],
        [ 0.23364672,  0.32846986,  0.6326616 , -0.09151491],
        [-1.05684875, -1.42349957, -1.3374081 ,  1.93054905],
        [-0.64311332, -2.00718505, -1.60691971,  1.43026965],
        [ 0.13988981,  0.28099617,  0.50405069, -0.44432248],
        [ 0.05787516,  0.09546232,  1.28959618, -0.14956287],
        [-0.42451288,  0.00777202,  0.83388789, -0.23143627],
        [ 2.15115809,  0.02331367, -0.83677661,  0.59475188],
        [ 2.38297868, -1.03423083, -0.66092546,  0.68480826],
        [ 1.95689934, -0.60490362, -0.8247449 , -0.00755088],
        [-0.51139536, -0.1408783 , -0.29199464, -0.09000482],
        [-0.61007558, -2.01992869, -1.49499303,  1.90725233],
        [ 1.88651323, -0.70172563, -0.54178926,  0.28770913],
        [ 0.47585128,  0.4080172 ,  0.01257563, -0.14044039],
        

## Hands-on


Design your own VQC to classify this data! Try to beat my record of 93.3\% accuracy.

In [4]:
NUM_WIRES = 4
NUM_PARAMS = 3
NUM_LAYERS = 2

In [5]:
weights = np.random.normal(size=(NUM_WIRES*NUM_LAYERS, NUM_PARAMS))

In [6]:
weights

tensor([[ 0.1049548 ,  1.66701038, -1.09682379],
        [-0.31850905,  0.20698853,  0.00185679],
        [ 1.45003148, -1.57998593,  0.43237273],
        [ 1.40336851, -0.64938225,  1.62909963],
        [-0.85178059, -0.29882222, -1.24790773],
        [-0.92005539,  0.4103234 ,  1.18440713],
        [ 0.62437989,  0.499178  ,  0.80402053],
        [-0.30086374, -0.16820966, -1.22348312]], requires_grad=True)

In [7]:
weights[0]

tensor([ 0.1049548 ,  1.66701038, -1.09682379], requires_grad=True)

In [27]:
dev = qml.device('default.qubit', wires=NUM_WIRES)

@qml.qnode(dev)
def vqc_model(data, weights):
    # YOUR CODE HERE
    # Design an ansatz for this data: choose how the data is encoded
    # and the gates that the weights are input into.
    qml.AngleEmbedding(data, wires=[0,1,2,3])
    qml.Rot(*weights[0], wires=0)
    qml.Rot(*weights[1], wires=1)
    qml.Rot(*weights[2], wires=2)
    qml.Rot(*weights[3], wires=3)

    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])

    qml.Rot(*weights[4], wires=0)
    qml.Rot(*weights[5], wires=1)
    qml.Rot(*weights[6], wires=2)
    qml.Rot(*weights[7], wires=3)

    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[3,0])

    return qml.expval(qml.PauliX(0), qml.PauliX)

In [28]:
print(qml.draw(vqc_model, expansion_strategy="device")(train_X[0], weights))

(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
0: ──RX(-0.77)──Rot(0.10,1.67,-1.10)─╭●───────╭X──Rot(-0.85,-0.30,-1.25)─╭●───────╭X─┤  <X>
1: ──RX(-1.89)──Rot(-0.32,0.21,0.00)─╰X─╭●────│───Rot(-0.92,0.41,1.18)───╰X─╭●────│──┤  <X>
2: ──RX(-1.03)──Rot(1.45,-1.58,0.43)────╰X─╭●─│───Rot(0.62,0.50,0.80)───────╰X─╭●─│──┤     
3: ──RX(1.47)───Rot(1.40,-0.65,1.63)───────╰X─╰●──Rot(-0.30,-0.17,-1.22)───────╰X─╰●─┤     


In [29]:
def loss(weights):
    # YOUR CODE HERE
    # Design a new loss function, or adapt the helper function

    loss_sum = 0.0

    for idx in range(train_X.shape[0]):
        point = train_X[idx]
        true_expval = train_y[idx]

        estimated_expval = vqc_model(point, weights)
        loss_sum += (estimated_expval - true_expval) ** 2

    return loss_sum / train_X.shape[0]

In [30]:
# Set up initial parameters, and write an optimization loop to train your model.

opt = qml.GradientDescentOptimizer(stepsize=0.1)

n_its = 100

loss_track = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(loss, weights)
    if it % 5 == 0:
        our_preds = make_predictions(train_X, vqc_model, weights)
        print(f"Loss at iteration {it} = {_loss}  Accuracy = {compute_accuracy(our_preds, train_y)}")
    loss_track.append(_loss)

(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(PauliX(wires=[0])), expval(PauliX(wires=[1])))
(expval(Pa

TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

In [ ]:
# def make_predictions(data, model, weights):
#     # YOUR CODE HERE
#     # Modify the make_predictions function based on how you set up your classifier.
#
#     return

After training, you can use the code below to evaluate the metrics.

In [ ]:
my_training_predictions = make_predictions(train_X, vqc_model, weights)
my_testing_predictions = make_predictions(test_X, vqc_model, weights)
                                          
training_accuracy = compute_accuracy(my_training_predictions, train_y)
testing_accuracy = compute_accuracy(my_testing_predictions, test_y)

print(f"Training accuracy = {training_accuracy}")
print(f"Testing accuracy = {testing_accuracy}")